## Using SMIRNOFF with Amber on BRD4:inhibitor complexes: Exporting parameterized complexes to Amber, Gromacs, and CHARMM

This example applies SMIRNOFF-format parameters to BRD4 inhibitors from the [living review on binding free energy benchmark systems](https://www.annualreviews.org/doi/abs/10.1146/annurev-biophys-070816-033654) by Mobley and Gilson. The BRD4 system comes from the [accompanying GitHub repository](https://github.com/MobleyLab/benchmarksets/tree/master/input_files/BRD4).

This example uses [ParmEd](http://parmed.github.io) to take a protein-ligand system parameterized with an alternate force field, and replace the force field used for the ligand with an OpenFF force field. This example is meant to illustrate how to apply parameters to a single ligand, but it's also easy to process many ligands.

In [2]:
# Retrieve protein and ligand files for BRD4 and a docked inhibitor from the benchmark systems GitHub repository
# https://github.com/MobleyLab/benchmarksets
import requests
repo_url = 'https://raw.githubusercontent.com/MobleyLab/benchmarksets/master/input_files/'
sources = {
    'system.prmtop' : repo_url + 'BRD4/prmtop-coords/BRD4-1.prmtop',
    'system.crd'   : repo_url + 'BRD4/prmtop-coords/BRD4-1.crds',
    'ligand.sdf'   : repo_url + 'BRD4/sdf/ligand-1.sdf',
    'system.pdb'   : repo_url + 'BRD4/prmtop-coords/BRD4-1.pdb',
    'ligand.pdb'   : repo_url + 'BRD4/pdb/ligand-1.pdb'
}
for (filename, url) in sources.items():
    r = requests.get(url)
    open(filename, 'w').write(r.text)

In [3]:
#Read AMBER to ParmEd Structure object
import parmed
in_prmtop = 'system.prmtop'
in_crd = 'system.crd'
structure = parmed.amber.AmberParm( in_prmtop, in_crd )


In [4]:
pieces = structure.split()

In [5]:
pieces

[(<AmberParm 2035 atoms; 121 residues; 2064 bonds; PBC (orthogonal); parametrized>,
  {0}),
 (<AmberParm 26 atoms; 1 residues; 28 bonds; PBC (orthogonal); parametrized>,
  {1}),
 (<AmberParm 1 atoms; 1 residues; 0 bonds; PBC (orthogonal); NOT parametrized>,
  {2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33}),
 (<AmberParm 1 atoms; 1 residues; 0 bonds; PBC (orthogonal); NOT parametrized>,
  {34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68}),
 (<AmberParm 3 atoms; 1 residues; 3 bonds; PBC (orthogonal); parametrized>,
  {69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
  

We use both a PDB file and an SDF file for the ligand.

In [8]:
# Create an OpenFF Molecule object from the ligand SDF fiel
from openforcefield.topology import Molecule, Topology
ligand_off_molecule = Molecule('ligand.sdf')

# Load the SMIRNOFF-format Parsley force field
from openforcefield.typing.engines.smirnoff import ForceField
force_field = ForceField('openff_unconstrained-1.0.0.offxml')

# Parametrize the ligand molecule by creating a Topology object from it
ligand_system = force_field.create_openmm_system(ligand_off_molecule.to_topology())

In [9]:
# Read in the coordinates of the ligand from the PDB file
from simtk.openmm.app import PDBFile
ligand_pdbfile = PDBFile('ligand.pdb')

# Convert OpenMM System object containing ligand parameters into a ParmEd Structure.
import parmed

# It should be able to work this way -- though this begins with a PDB file which is not as elegant
#ligand_structure = parmed.openmm.load_topology(ligand_pdbfile.topology,
#                                                ligand_system,
#                                                xyz=ligand_pdbfile.positions)

# It should also be able to work this way, which does not use a PDB file
ligand_off_topology = Topology.from_molecules([ligand_off_molecule])
ligand_structure = parmed.openmm.load_topology(ligand_off_topology.to_openmm(),
                                                ligand_system,
                                                xyz=pieces[1][0].positions)

In [10]:
ligand_structure.save('tmp.prmtop', overwrite=True)
ligand_structure.save('tmp.inpcrd', overwrite=True)
ligand_structure.save('tmp.gro', overwrite=True)
ligand_structure.save('tmp.top', overwrite=True)

In [11]:
from openforcefield.topology import Molecule
from openforcefield.topology import Topology
from openforcefield.typing.engines.smirnoff import ForceField




# Check how many atoms in ligand loaded directly
n_atoms_new = 0
for atom in ligand_off_molecule.atoms:
    #Print out elements as a quick check they are in teh same order
    #print(atom.element)
    n_atoms_new +=1
    
parmed_lig, n_copies = pieces[1]

# Check how many atoms in ligand from prmtop file
n_atoms = 0
for atom in parmed_lig:
    # Print elements as quick check
    #print(atom.element)
    n_atoms +=1

# Print out error message if number of atoms doesn't match
if n_atoms != n_atoms_new:
    print("Error: Number of atoms in input ligand doesn't match number extracted from prmtop file.")

In [12]:
# Combine receptor and ligand structures

#Combine the protein and the ligand
complex_structure = pieces[0][0] + ligand_structure 

#Ultimately we'd like to combine the protein and ligand and the water, which would probably be something like this:
#complex_structure = pieces[0][0] + ligand_structure + pieces[2][0]*len(pieces[2][1])

# For debug purposes we could try just this
#complex_structure = ligand_structure

# Export the Structure to AMBER files
complex_structure.save('complex.prmtop', overwrite=True)
complex_structure.save('complex.inpcrd', overwrite=True)

# Export the Structure to Gromacs files
complex_structure.save('complex.gro', overwrite=True)
complex_structure.save('complex.top', overwrite=True)

AmberError: FLAG LENNARD_JONES_ACOEF has 171 elements; expected 105